In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import from_avro, to_avro
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json
from pyspark.ml import Pipeline,PipelineModel

In [2]:
#Spark Session creation configured to interact with Kfka and MongoDB
spark = SparkSession.builder.appName("pyspark-notebook").\
config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-avro_2.12:3.0.0").\
getOrCreate()

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3a19a85e-382f-449a-9213-271917f69a33;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
	

In [3]:
#Read schema file and create schema of string type
json_schema = ''
with open("Text-generate/schema/out/blackfriday.json") as f:
    new_schema = StructType.fromJson(json.load(f))
    json_schema = new_schema.simpleString()

In [4]:
#Read data from Kafka topic
json_tweets = spark\
  .readStream\
  .format("kafka")\
  .option("kafka.bootstrap.servers", "192.168.56.1:9092")\
  .option("subscribe", "blackfriday")\
  .option("startingOffsets", "earliest")\
  .option("failOnDataLoss", "false") \
  .load()\
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [5]:
#Refine raw data red from Kafka topic
refined_tweets = json_tweets\
        .select(from_json("value", json_schema)\
        .alias("data"))\
        .where("data.language='en'and data.creation_date is not null and data.text is not null")\
        .select("data.text",
                from_unixtime(col("data.timestamp")/1000,'yyyy-MM-dd HH:mm:ss').alias("timestamp")) #Translate milliseconds to UTC timestamp
refined_tweets = refined_tweets.withColumn('text', regexp_replace('text', r'http\S+', ''))
refined_tweets = refined_tweets.withColumn('text', regexp_replace('text', '@\w+', ''))
refined_tweets = refined_tweets.withColumn('text', regexp_replace('text', '#', ''))
refined_tweets = refined_tweets.withColumn('text', regexp_replace('text', 'RT', ''))
refined_tweets = refined_tweets.withColumn('text', regexp_replace('text', ':', ''))

24/12/08 15:22:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
# Xuất dữ liệu refined_tweets thành file text
query = refined_tweets.select("text").writeStream \
    .format("text") \
    .option("path", "Text-generate/output/tweets_txt") \
    .option("checkpointLocation", "Text-generate/output/checkpoints_txt") \
    .start()

# Đợi luồng dữ liệu kết thúc (tùy chọn, có thể để chạy song song)
query.awaitTermination()


In [9]:
# Đọc dữ liệu từ tệp .txt
with open('Text-generate/output/tweets_txt/outputdata.txt', 'r') as file:
    text_data = file.read()

# Kiểm tra một vài dòng
print(text_data[:500])  # In 500 ký tự đầu tiên


Black Friday 50% OFF SALE ends Midnight Sunday PST 💥💥💥  
Black Friday Weekend Sale
ENDS TODAY 8th DECEMBER

20% discount off all Sale items. 



MHHSBD 
Starting Wednesday 11.23 our first ever BLACK FRIDAY SALE. With up to 70% off sitewide. Sign up now for early access  
Black Friday Sale Time!

Is on until 28 Nov 22~ Link below!~ 
Black Friday Weekend Sale
ENDS TODAY 8th DECEMBER

20% discount off all Sale items. 



MHHSBD 
BLACK FRIDAY SALE IS LIVE🦋 up to 40% off worldwide !!!

 
65% Off Blac
